Database Project
==============================

This is our first example of an in class project.  In this project I will be going through the steps
that are listed below to help us to create a new system that can act like a database.  This will work
by using the following items. 

1. Functions
  * especially lambdas
2. Generators
3. Decorators
4. Set Operations

### Permissions

We will implement the idea of a permission for each of our database functions, we will do this by creating a 
decorator that will look at our global user and if the user has the required permission they can run it.  

    @authorize('Admin')
    def delete_query():
        pass

### Streams

We will be able to create queries that can "join" by the fact that they will be each returning a generator
(not a set list) that we can then use to create other generators.  

    def select_query(where):
        return (x for x in table if where(x))

    def join(query1, field1, query2, field2):
        return ((q1, q2) for q1 in query1 for q2 in query2 if q1[field1] == q2[field2])

## Helper functions

At this point I am going to create a few helper functions that will be used for this system. This
will help in data generation and other operations.  

In [ ]:
import random

## First Steps

For the first step we are going to define our global values, this includes our tables and our user info.  

We will also want to create some test data that we will put into our created tables.  

In [ ]:
# Student Table (id*, name*)
student_table = []
# Class Table (id*, name*, time)
class_table = []
# Assignment Table (id*, name*, total_score*)
assignment_table = []
# Students Classes Table (student_id*, class_id*, grade)
student_class_table = []
# Student Assignment Table (student_id*, class_id*, assignment_id*, score)
student_assignment_table = []

In [ ]:
users_list = {
    'teacher': {'Admin', 'User'},
    'student': {'User'},
}

user = 'student'

## Second step

At this point we are going to create a few helper functions that we can use to query data

## Queries

* Students who are in Dance or Finance.  
* Students in Dance and Computer Science.
* Average Score for Eassay assignment in class Computer Science
